In [ ]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
# pip install mysql-connector-python
import mysql.connector

# Creating connection object
mydb_conn = mysql.connector.connect(
    host = "127.0.0.1",
    user = "root",
    password = "Hana@@##1234",
    port=3306,
    database='library'
)

# Printing the connection object 
print(mydb_conn)

In [43]:
cursor = mydb_conn.cursor()
cursor

In [44]:
cursor.execute("SELECT * FROM books")
result = cursor.fetchall()
result

[(1,
  'Walden, and On The Duty Of Civil Disobedience',
  'Henry David Thoreau',
  1849,
  3100471153),
 (2, 'The Scarlet Letter', 'Nathaniel Hawthorne', 1850, 4589408958),
 (3,
  'The Adventures of Sherlock Holmes',
  'Arthur Conan Doyle',
  1892,
  2411676280),
 (4, 'The Adventures of Tom Sawyer', 'Mark Twain', 1876, 7813216705),
 (5, 'David Copperfield', 'Charles Dickens', 1850, 8821265677),
 (6, 'A Modest Proposal', 'Jonathan Swift', 1729, 9675136364),
 (7, 'The Sign of the Four', 'Arthur Conan Doyle', 1890, 9615072124),
 (8,
  'The Importance of Being Earnest: A Trivial Comedy for Serious People',
  'Oscar Wilde',
  1895,
  4733732465),
 (9, "Uncle Tom's Cabin", 'Harriet Beecher Stowe', 1852, 1874764443),
 (10, 'The Count of Monte Cristo', 'Alexandre Dumas', 1884, 1799238621),
 (11, 'The Picture of Dorian Gray', 'Oscar Wilde', 1890, 2855934983),
 (12, 'Dracula', 'Bram Stoker', 1897, 8232533734),
 (13, 'Oliver Twist', 'Charles Dickens', 1839, 3071366111),
 (14,
  'The Awakening, an

In [59]:
# find the number of availalbe copies of the book (Dracula)

cursor.execute("""
               
SELECT 
    (SELECT COUNT(*) FROM Books WHERE Title = 'Dracula') 
    - 
    (SELECT COUNT(*) FROM Loans l 
     JOIN Books b ON l.BookID = b.BookID 
     WHERE b.Title = 'Dracula' AND l.ReturnedDate IS NULL) 
    AS AvailableCopies;

""")


cursor.fetchall() 

[(2,)]

In [63]:
# check total copies of the book
cursor.execute("""
               
SELECT COUNT(*) AS TotalCopies 
FROM Books 
WHERE Title = 'Dracula';

""")


cursor.fetchall()           

[(3,)]

In [47]:
# current total loans of the book

cursor.execute("""
               
SELECT COUNT(*) AS TotalLoans
FROM Loans l
JOIN Books b ON l.BookID = b.BookID
WHERE b.Title = 'Dracula' AND l.ReturnedDate IS NULL;
""")


cursor.fetchall()

[(1,)]

In [48]:
# total available books of dracula 

cursor.execute("""

SELECT COUNT(*) AS AvailableCopies
FROM Books b
LEFT JOIN Loans l ON b.BookID = l.BookID AND l.ReturnedDate IS NULL
WHERE b.Title = 'Dracula' AND l.LoanID IS NULL;
""")

cursor.fetchall()

[(2,)]

In [61]:
# Add new books to the library

cursor.execute("""

INSERT INTO books (BookID, Title, Author, Published, Barcode)
VALUES (205,'To jlol a Mockingoird', 'Horoer Lpe', 7860, '928096112');
""")

cursor.fetchall()


[]

In [20]:
# Check out Books: books(4043822646, 2855934983)

cursor.execute("""

SELECT 
    (SELECT COALESCE(MAX(LoanID), 0) + 1 FROM Loans) AS LoanID,
    b.BookID,
    p.PatronID,
    '2020-08-25',
    '2020-09-08'
FROM Books b
JOIN Patrons p ON p.Email = 'jvaan@wisdompets.com'
WHERE b.Barcode IN (4043822646, 2855934983);
""")

cursor.fetchall()


[(2001, 93, 50, '2020-08-25', '2020-09-08'),
 (2001, 11, 50, '2020-08-25', '2020-09-08')]

In [21]:
# Check books for Due back

cursor.execute("""
SELECT 
    l.LoanID,
    b.Title AS BookTitle,
    p.FirstName || ' ' || p.LastName AS PatronName,
    p.Email AS PatronEmail,
    l.LoanDate,
    l.DueDate,
    l.ReturnedDate
FROM Loans l
JOIN Books b ON l.BookID = b.BookID
JOIN Patrons p ON l.PatronID = p.PatronID
WHERE l.ReturnedDate IS NULL
ORDER BY l.DueDate ASC;

""")

cursor.fetchall()

[(1800,
  'Three Men in a Boat (To Say Nothing of the Dog)',
  0,
  'cdurdy2p@hplussport.com',
  '2020-02-08',
  '2020-02-22',
  None),
 (1864,
  'Jane Eyre: An Autobiography',
  0,
  'mdomney0@wisdompets.com',
  '2020-03-24',
  '2020-04-07',
  None),
 (1911,
  'Jane Eyre: An Autobiography',
  0,
  'cmcgilroy11@landonhotel.com',
  '2020-04-24',
  '2020-05-08',
  None),
 (1948,
  'Sense and Sensibility',
  0,
  'aofihillie1f@wisdompets.com',
  '2020-05-17',
  '2020-05-31',
  None),
 (1983,
  'David Copperfield',
  0,
  'gholyland2f@red30design.com',
  '2020-06-17',
  '2020-07-01',
  None),
 (1984,
  'An Occurrence at Owl Creek Bridge',
  0,
  'vmclarnon24@orangevalleycaa.org',
  '2020-06-18',
  '2020-07-02',
  None),
 (1985,
  "Grimms' Fairy Tales",
  0,
  'rwalkowski1i@kinetecoinc.com',
  '2020-06-18',
  '2020-07-02',
  None),
 (1986,
  'Walden, and On The Duty Of Civil Disobedience',
  0,
  'ctodeoe@wisdompets.com',
  '2020-06-18',
  '2020-07-02',
  None),
 (1987,
  'Also sprach Zarat

In [22]:
# generate a report of books due back on July 13, 2020
cursor.execute("""
SELECT 
    l.LoanID,
    b.Title AS BookTitle,
    b.Barcode AS BookBarcode,
    p.FirstName || ' ' || p.LastName AS PatronName,
    p.Email AS PatronEmail,
    l.LoanDate,
    l.DueDate
FROM Loans l
JOIN Books b ON l.BookID = b.BookID
JOIN Patrons p ON l.PatronID = p.PatronID
WHERE l.DueDate = '2020-07-13'
ORDER BY p.LastName, p.FirstName; 

""")

cursor.fetchall()

[(1997,
  'The Prophet',
  9013040583,
  0,
  'lawmack1a@samoca.org',
  '2020-06-29',
  '2020-07-13'),
 (1998,
  'Second Treatise of Government',
  5647388501,
  0,
  'adinjesh@samoca.org',
  '2020-06-29',
  '2020-07-13'),
 (1999,
  'Dracula',
  8730298424,
  0,
  'ckoppe1m@landonhotel.com',
  '2020-06-29',
  '2020-07-13'),
 (1996,
  'The Count of Monte Cristo',
  1799238621,
  0,
  'ksimonian2q@landonhotel.com',
  '2020-06-29',
  '2020-07-13')]

In [23]:
# with patron contact information
cursor.execute("""
SELECT 
    l.LoanID,
    b.Title AS BookTitle,
    b.Barcode AS BookBarcode,
    p.FirstName || ' ' || p.LastName AS PatronName,
    p.Email AS PatronEmail,
    p.PatronID,
    l.LoanDate,
    l.DueDate
FROM Loans l
JOIN Books b ON l.BookID = b.BookID
JOIN Patrons p ON l.PatronID = p.PatronID
WHERE l.DueDate = '2020-07-13'
ORDER BY p.LastName, p.FirstName;
""")

cursor.fetchall()

[(1997,
  'The Prophet',
  9013040583,
  0,
  'lawmack1a@samoca.org',
  47,
  '2020-06-29',
  '2020-07-13'),
 (1998,
  'Second Treatise of Government',
  5647388501,
  0,
  'adinjesh@samoca.org',
  18,
  '2020-06-29',
  '2020-07-13'),
 (1999,
  'Dracula',
  8730298424,
  0,
  'ckoppe1m@landonhotel.com',
  59,
  '2020-06-29',
  '2020-07-13'),
 (1996,
  'The Count of Monte Cristo',
  1799238621,
  0,
  'ksimonian2q@landonhotel.com',
  99,
  '2020-06-29',
  '2020-07-13')]

In [30]:
#Return books to the library (which have barcode=6435968624) and return this book at this date(2020-07-05)
cursor.execute("""
SET SQL_SAFE_UPDATES = 0;
UPDATE Loans 
SET ReturnedDate = '2020-07-05' 
WHERE BookID = (SELECT BookID FROM Books WHERE Barcode = 6435968624) 
AND ReturnedDate IS NULL;

""")

cursor.fetchall()

[]

In [ ]:
#Encourage Patrons to check out books as follows: 

In [50]:

#1# Recommend Books Based on Reading History

cursor.execute("""
SELECT DISTINCT b.Title, b.Author
FROM Books b
WHERE b.BookID NOT IN (
    SELECT l.BookID FROM Loans l 
    WHERE l.PatronID = (SELECT PatronID FROM Patrons WHERE Email = 'patron@example.com')
)
ORDER BY RAND()
LIMIT 5;
""")
cursor.fetchall()



[('Also sprach Zarathustra', 'Friedrich Wilhelm Nietzsche'),
 ('Emma', 'Jane Austen'),
 ('The Secret Garden', 'Frances Hodgson Burnett'),
 ('The War of the Worlds', 'H. G. Wells'),
 ("Pascal's Pensées", 'Blaise Pascal')]

In [51]:
#2# Promote Most Borrowed Books
cursor.execute("""
SELECT b.Title, b.Author, COUNT(l.BookID) AS LoanCount
FROM Loans l
JOIN Books b ON l.BookID = b.BookID
GROUP BY b.BookID
ORDER BY LoanCount DESC
LIMIT 5;

""")

cursor.fetchall()



[('The War of the Worlds', 'H. G. Wells', 20),
 ('Frankenstein; Or, The Modern Prometheus',
  'Mary Wollstonecraft Shelley',
  20),
 ('The Scarlet Letter', 'Nathaniel Hawthorne', 18),
 ('The Awakening, and Selected Short Stories', 'Kate Chopin', 17),
 ('Frankenstein; Or, The Modern Prometheus',
  'Mary Wollstonecraft Shelley',
  17)]

In [52]:
#3# Remind Patrons Who Haven't Borrowed Recently
cursor.execute("""
SELECT p.FirstName, p.LastName, p.Email
FROM Patrons p
WHERE p.PatronID NOT IN (
    SELECT DISTINCT l.PatronID FROM Loans l
    WHERE l.LoanDate >= DATE_SUB(CURDATE(), INTERVAL 6 MONTH)
);
""")

cursor.fetchall()





[('Maggi', 'Domney', 'mdomney0@wisdompets.com'),
 ('Javier', 'Dawks', 'jdawks1@red30design.com'),
 ('Aleen', 'Fasey', 'afasey2@kinetecoinc.com'),
 ('Taylor', 'Jenkins', 'tjenkins@rouxacademy.org'),
 ('Imogen', 'Kabsch', 'ikabsch@landonhotel.com'),
 ('Don', 'Weingarten', 'dweingarten@kinetecoinc.com'),
 ('Cammi', 'Kynett', 'ckynett6@orangevalleycaa.org'),
 ('Steffie', 'Kleis', 'skleis7@wisdompets.com'),
 ('Carilyn', 'Calver', 'ccalver8@samoca.org'),
 ('Barbara-anne', 'Sweet', 'bsweet9@samoca.org'),
 ('Faina', 'Nelles', 'fnellesa@red30design.com'),
 ('Orton', 'Stavers', 'ostaversb@orangevalleycaa.org'),
 ('Jobina', 'McKern', 'jmckernc@rouxacademy.org'),
 ('Paco', 'Yarranton', 'pyarrantond@rouxacademy.org'),
 ('Chance', 'Todeo', 'ctodeoe@wisdompets.com'),
 ('Carla', 'Swinfen', 'cswinfenf@kinetecoinc.com'),
 ('Lani', 'Over', 'loverg@wisdompets.com'),
 ('Anthe', 'Dinjes', 'adinjesh@samoca.org'),
 ('Amby', 'Harber', 'aharberi@hplussport.com'),
 ('Jacinta', 'Vallis', 'jvallisj@kinetecoinc.com

In [54]:
#4# Recommend New Arrivals
cursor.execute("""
SELECT Title, Author, Published
FROM Books
WHERE Published >= YEAR(CURDATE()) - 1
ORDER BY Published DESC
LIMIT 5;

""")

cursor.fetchall()

[]

In [55]:
#generate a report of showing 10 patrons who have
cursor.execute("""
SELECT p.PatronID, p.FirstName, p.LastName, p.Email, 
       COUNT(l.LoanID) AS TotalLoans
FROM Patrons p
LEFT JOIN Loans l ON p.PatronID = l.PatronID
GROUP BY p.PatronID, p.FirstName, p.LastName, p.Email
ORDER BY TotalLoans ASC
LIMIT 10;

""")

cursor.fetchall()


[(83, 'Caril', 'Matejic', 'cmatejic2a@orangevalleycaa.org', 9),
 (69, 'Alaster', 'Ruggles', 'aruggles1w@red30design.com', 9),
 (73, 'Merle', 'Sukbhans', 'msukbhans20@orangevalleycaa.org', 11),
 (64, 'Norby', 'Sleight', 'nsleight1r@kinetecoinc.com', 12),
 (94, 'Curr', 'Fittall', 'cfittall2l@orangevalleycaa.org', 12),
 (50, 'Jack', 'Vaan', 'jvaan@wisdompets.com', 13),
 (7, 'Cammi', 'Kynett', 'ckynett6@orangevalleycaa.org', 14),
 (3, 'Aleen', 'Fasey', 'afasey2@kinetecoinc.com', 14),
 (12, 'Orton', 'Stavers', 'ostaversb@orangevalleycaa.org', 14),
 (16, 'Carla', 'Swinfen', 'cswinfenf@kinetecoinc.com', 14)]

In [56]:
# Find books to feature for an event create a list of books from 1890s that ar currently available 
cursor.execute("""
SELECT b.Title, b.Author, b.Published, b.Barcode 
FROM Books b
LEFT JOIN Loans l ON b.BookID = l.BookID AND l.ReturnedDate IS NULL
WHERE b.Published BETWEEN 1890 AND 1899
AND l.LoanID IS NULL;
""")

cursor.fetchall()

[('The Adventures of Sherlock Holmes', 'Arthur Conan Doyle', 1892, 2411676280),
 ('The Sign of the Four', 'Arthur Conan Doyle', 1890, 9615072124),
 ('The Importance of Being Earnest: A Trivial Comedy for Serious People',
  'Oscar Wilde',
  1895,
  4733732465),
 ('The Picture of Dorian Gray', 'Oscar Wilde', 1890, 2855934983),
 ('Dracula', 'Bram Stoker', 1897, 8232533734),
 ('The Awakening, and Selected Short Stories',
  'Kate Chopin',
  1899,
  3103139173),
 ('The Time Machine', 'H. G. Wells', 1895, 9610930110),
 ('Les Misérables', 'Victor Hugo', 1895, 8252831254),
 ('The Jungle Book', 'Rudyard Kipling', 1894, 3655763446),
 ('Heart of Darkness', 'Joseph Conrad', 1899, 9334362486),
 ('The Adventures of Sherlock Holmes', 'Arthur Conan Doyle', 1892, 6117891368),
 ('Heart of Darkness', 'Joseph Conrad', 1899, 2229312049),
 ('The Yellow Wallpaper', 'Charlotte Perkins Gilman', 1892, 7476179474),
 ('The Yellow Wallpaper', 'Charlotte Perkins Gilman', 1892, 2503974912),
 ('An Occurrence at Owl Cr

In [57]:
#create a report to show how many books were published each year.
cursor.execute("""
SELECT Published AS Year, COUNT(*) AS TotalBooks 
FROM Books 
GROUP BY Published 
ORDER BY Published ASC;
""")

cursor.fetchall()

[(1375, 2),
 (1532, 3),
 (1605, 2),
 (1670, 2),
 (1689, 2),
 (1726, 3),
 (1729, 3),
 (1759, 3),
 (1791, 3),
 (1811, 3),
 (1813, 3),
 (1815, 3),
 (1818, 3),
 (1839, 3),
 (1847, 6),
 (1849, 3),
 (1850, 6),
 (1851, 3),
 (1852, 3),
 (1854, 3),
 (1858, 3),
 (1859, 6),
 (1861, 3),
 (1865, 3),
 (1869, 6),
 (1876, 3),
 (1878, 3),
 (1879, 3),
 (1880, 3),
 (1883, 3),
 (1884, 6),
 (1886, 3),
 (1887, 3),
 (1889, 3),
 (1890, 9),
 (1892, 6),
 (1894, 3),
 (1895, 9),
 (1896, 3),
 (1897, 3),
 (1898, 6),
 (1899, 5),
 (1900, 3),
 (1901, 3),
 (1902, 2),
 (1905, 3),
 (1906, 2),
 (1908, 3),
 (1911, 6),
 (1913, 2),
 (1914, 3),
 (1915, 3),
 (1920, 3),
 (1921, 2),
 (1922, 5),
 (1923, 3),
 (1950, 1),
 (1960, 1)]

In [58]:
# create a report to show 5 most popular Books to check out
cursor.execute("""
SELECT 
    b.Title, 
    b.Author, 
    b.Published, 
    COUNT(l.LoanID) AS TotalTimesOfLoans 
FROM Books b
JOIN Loans l ON b.BookID = l.BookID
GROUP BY b.Title, b.Author, b.Published
ORDER BY TotalTimesOfLoans DESC
LIMIT 5;
""")

cursor.fetchall()


[('Frankenstein; Or, The Modern Prometheus',
  'Mary Wollstonecraft Shelley',
  1818,
  53),
 ('The War of the Worlds', 'H. G. Wells', 1898, 43),
 ('The Awakening, and Selected Short Stories', 'Kate Chopin', 1899, 43),
 ('The Scarlet Letter', 'Nathaniel Hawthorne', 1850, 42),
 ('The Picture of Dorian Gray', 'Oscar Wilde', 1890, 39)]